In [21]:
from bbq.decoder import bp_osd
import numpy as np
from ldpc import BpDecoder, bposd_decoder
from bbq.polynomial import Polynomial, Monomial
from bbq.bbq_code import BivariateBicycle
from bbq.field import Field
import matplotlib.pyplot as plt
import json
import datetime
import galois
from ldpc import bposd_decoder

In [23]:
my_bp_method = "ms"
my_max_iter = 300
my_osd_method = "osd_cs"
my_osd_order = 0
my_ms_scaling_factor = 0

In [26]:
def simulate_ldpc(field, h, l, max_iter, num_failures, physical_error, my_bp_method = "ms", my_max_iter = 300, my_osd_method = "osd_cs", my_osd_order = 0, my_ms_scaling_factor = 0):

    results = []

    for p in physical_error:

        failures = 0
        num_trials = 0

        n_qudits = h.shape[1]

        bpd = bposd_decoder(
            h,
            error_rate = float(p),
            max_iter = my_max_iter,
            bp_method = my_bp_method,
            ms_scaling_factor = my_ms_scaling_factor,
            osd_method = my_osd_method,
            osd_order = my_osd_order
            )

        while failures < num_failures:
            # Generate syndrome
            error = np.zeros(n_qudits, dtype=int)
            error_mask = np.random.rand(n_qudits) < p
            for i in np.where(error_mask)[0]:
                error[i] = np.random.randint(1, field.p)
            syndrome = (h @ error) % field.p

            # Decode
            # guessed_error, decoder_success, bp_success, posterior = belief_propagation(field, h, syndrome, x_prior, max_iter, debug=True)
            bpd.decode(syndrome)
            guessed_error = bpd.osdw_decoding
            error_difference = (error - guessed_error) % field.p
            logical_effect = (np.array(l) @ error_difference) % field.p

            # Check success
            # if np.any(logical_effect != 0) or not decoder_success:
            if np.any(logical_effect != 0):
                failures += 1
                print(f'Found {failures} / {num_failures}, with num_trials : {num_trials}')
            elif num_trials % 100 == 0:
                print(f'UPDATE: Found {failures} / {num_failures}, with num_trials : {num_trials}')

            num_trials += 1

        results.append(num_trials)

        print(f'Finished p={p} with num_trials={num_trials}')
        print('------------------------------------------------------------------')
    return results

In [29]:
results = np.flip(simulate_ldpc(field, h72, l72, max_iter, 20, physical_error))

Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 1
Found 3 / 20, with num_trials : 2
Found 4 / 20, with num_trials : 3
Found 5 / 20, with num_trials : 4
Found 6 / 20, with num_trials : 5
Found 7 / 20, with num_trials : 6
Found 8 / 20, with num_trials : 7
Found 9 / 20, with num_trials : 8
Found 10 / 20, with num_trials : 9
Found 11 / 20, with num_trials : 10
Found 12 / 20, with num_trials : 11
Found 13 / 20, with num_trials : 12
Found 14 / 20, with num_trials : 13
Found 15 / 20, with num_trials : 14
Found 16 / 20, with num_trials : 15
Found 17 / 20, with num_trials : 16
Found 18 / 20, with num_trials : 17
Found 19 / 20, with num_trials : 18
Found 20 / 20, with num_trials : 19
Finished p=0.19952623149688797 with num_trials=20
------------------------------------------------------------------
Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 1
Found 3 / 20, with num_trials : 3
Found 4 / 20, with num_trials : 5
Found 5 / 20, with num_trials : 6
Found 6 / 20

In [31]:
results

array([103, 161, 254, 129, 116,  54,  41,  24,  22,  20])

In [32]:
results = np.flip(simulate_ldpc(field, h108, l108, max_iter, 20, physical_error))

c:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\bbq_venv\Lib\site-packages\ldpc\_legacy_ldpc_v1\_legacy_bposd_decoder.py:45: UserWarning: This is the old syntax for the `bposd_decoder` from `ldpc v1`. Use the `BpOsdDecoder` class from `ldpc v2` for additional features.
  warnings.warn(


Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 1
Found 3 / 20, with num_trials : 2
Found 4 / 20, with num_trials : 3
Found 5 / 20, with num_trials : 4
Found 6 / 20, with num_trials : 5
Found 7 / 20, with num_trials : 6
Found 8 / 20, with num_trials : 7
Found 9 / 20, with num_trials : 8
Found 10 / 20, with num_trials : 9
Found 11 / 20, with num_trials : 10
Found 12 / 20, with num_trials : 11
Found 13 / 20, with num_trials : 12
Found 14 / 20, with num_trials : 13
Found 15 / 20, with num_trials : 14
Found 16 / 20, with num_trials : 15
Found 17 / 20, with num_trials : 16
Found 18 / 20, with num_trials : 18
Found 19 / 20, with num_trials : 19
Found 20 / 20, with num_trials : 20
Finished p=0.19952623149688797 with num_trials=21
------------------------------------------------------------------
Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 1
Found 3 / 20, with num_trials : 2
Found 4 / 20, with num_trials : 3
Found 5 / 20, with num_trials : 4
Found 6 / 20

In [22]:
def simulate(field, h, l, max_iter, num_failures, physical_error):

    results = []

    for p in physical_error:

        failures = 0
        num_trials = 0

        while failures < num_failures:
            # Generate syndrome
            n_qudits = h.shape[1]
            error = np.zeros(n_qudits, dtype=int)
            error_mask = np.random.rand(n_qudits) < p
            for i in np.where(error_mask)[0]:
                error[i] = np.random.randint(1, field.p)
            syndrome = (h @ error) % field.p

            # Construct error probability
            channel_prob_x = np.ones(n_qudits) * p

            x_prior = np.zeros((len(channel_prob_x), field.p), dtype=float)

            for i, prob in enumerate(channel_prob_x):
                x_prior[i, 0] = 1 - prob
                for j in range(1, field.p):
                    x_prior[i, j] = prob / (field.p - 1)

            # Decode
            # guessed_error, decoder_success, bp_success, posterior = belief_propagation(field, h, syndrome, x_prior, max_iter, debug=True)
            guessed_error, decoder_success, bp_success, posterior = bp_osd(field, h, syndrome, x_prior, max_iter, order=0, debug=True)
            error_difference = (error - guessed_error) % field.p
            logical_effect = (np.array(l) @ error_difference) % field.p

            # Check success
            # if np.any(logical_effect != 0) or not decoder_success:
            if np.any(logical_effect != 0):
                failures += 1
                print(f'Found {failures} / {num_failures}, with num_trials : {num_trials}')
            elif num_trials % 100 == 0:
                print(f'UPDATE: Found {failures} / {num_failures}, with num_trials : {num_trials}')

            num_trials += 1

        results.append(num_trials)

        print(f'Finished p={p} with num_trials={num_trials}')
        print('------------------------------------------------------------------')
    return results

In [3]:
def circ_simulate(field, bb, syndrome_circ, num_cycles, max_iter, num_failures, physical_error):

    results = []
    qubits_dict = bb.qubits_dict
    data_qubits = bb.data_qubits
    z_checks = bb.z_checks
    z_logicals = bb.z_logicals
    first_logical_row = bb.l * bb.m * (num_cycles + 2)
    k = len(z_logicals)

    for p in physical_error:

        error_rates = {'Meas': p, 'Prep': p, 'Idle': p, 'CNOT': p}
        # this step longgg
        print(f'Constructing decoding matrix for p={p}...')
        hx_eff, short_hx_eff, hz_eff, short_hz_eff, channel_prob_x, channel_prob_z = bb.construct_decoding_matrix(syndrome_circ, error_rates, num_cycles)
        hx_eff, short_hx_eff = hx_eff.toarray(), short_hx_eff.toarray()
        print(f'Constructed decoding matrix for p={p}')

        failures = 0
        num_trials = 0

        # Construct error probability
        x_prior = np.zeros((len(channel_prob_x), field.p), dtype=float)

        for i, prob in enumerate(channel_prob_x):
            x_prior[i, 0] = 1 - prob
            for j in range(1, field.p):
                x_prior[i, j] = prob / (field.p - 1)

        while failures < num_failures:

            # Generate noisy circuit
            noisy_circ, err_cnt = bb._generate_noisy_circuit(syndrome_circ * num_cycles, error_rates)

            x_syndrome_history, x_state, x_syndrome_map, x_err_count = bb._simulate_x_circuit(noisy_circ + syndrome_circ + syndrome_circ)
            x_state_data_qubits = [x_state[qubits_dict[qubit]] for qubit in data_qubits]
            x_syndrome_final_logical = (np.array(z_logicals) @ x_state_data_qubits) % field.p
            # Syndrome sparsification
            x_syndrome_history_copy = x_syndrome_history.copy()
            for check in z_checks:
                pos = x_syndrome_map[check]
                assert len(pos) == num_cycles + 2
                for row in range(1, num_cycles + 2):
                    x_syndrome_history[pos[row]] += x_syndrome_history_copy[pos[row-1]]
            x_syndrome_history %= field.p

            # Decode
            guessed_error, decoder_success, bp_success, posterior = bp_osd(field, short_hx_eff, x_syndrome_history, x_prior, max_iter, order=0, debug=True)

            # Check success
            x_syndrome_history_augmented_guessed = (hx_eff @ guessed_error) % field.p
            x_syndrome_final_logical_guessed = x_syndrome_history_augmented_guessed[first_logical_row: first_logical_row + k]
            x_success = np.array_equal(x_syndrome_final_logical_guessed, x_syndrome_final_logical)
            # if np.any(logical_effect != 0) or not decoder_success:
            if not x_success:
                failures += 1
                print(f'Found {failures} / {num_failures}, with num_trials : {num_trials}')
            elif num_trials % 100 == 0:
                print(f'UPDATE: Found {failures} / {num_failures}, with num_trials : {num_trials}')

            num_trials += 1

        results.append(num_trials)

        print(f'Finished p={p} with num_trials={num_trials}')
        print('------------------------------------------------------------------')
    return results

In [4]:
field = Field(2)
a = Polynomial(field, np.array([[1, 0], [1, 0]]))
b = Polynomial(field, np.array([[1, 1], [0, 0]]))
bb2 = BivariateBicycle(a, b, 3, 3, 1)
h2 = bb2.hx
l2 = bb2.x_logicals
bb25 = BivariateBicycle(a, b, 5, 5, 1)
h25 = bb25.hx
l25 = bb25.x_logicals
bb27 = BivariateBicycle(a, b, 7, 7, 1)
h27 = bb27.hx
l27 = bb27.x_logicals

field3 = Field(3)
a = Polynomial(field3, np.array([[1, 0], [-1, 0]]))
b = Polynomial(field3, np.array([[1, -1], [0, 0]]))
bb3 = BivariateBicycle(a, b, 3, 3, 1)
h3 = bb3.hx
l3 = bb3.x_logicals
bb35 = BivariateBicycle(a, b, 5, 5, 1)
h35 = bb35.hx
l35 = bb35.x_logicals
bb37 = BivariateBicycle(a, b, 7, 7, 1)
h37 = bb37.hx
l37 = bb37.x_logicals

field5 = Field(5)
a = Polynomial(field5, np.array([[1, 0], [-1, 0]]))
b = Polynomial(field5, np.array([[1, -1], [0, 0]]))
bb5 = BivariateBicycle(a, b, 3, 3, 1)
h5 = bb5.hx
l5 = bb5.x_logicals
bb55 = BivariateBicycle(a, b, 5, 5, 1)
h55 = bb55.hx
l55 = bb55.x_logicals
bb57 = BivariateBicycle(a, b, 7, 7, 1)
h57 = bb57.hx
l57 = bb57.x_logicals

In [14]:
max_iter = 300
num_failures = 3
long_physical_error = np.logspace(-2.7, -1, 10)
physical_error = np.flip(np.logspace(-1.7, -0.7, 10))
ext_physical_error = np.flip(np.logspace(-1, -0.8, 10))

In [6]:
x, y = Monomial(field, "x"), Monomial(field, "y")

a = y**3 + x + x**2
b = x**3 + y + y**2

bbq_72 = BivariateBicycle(a, b, 6, 6, 1)
h72 = bbq_72.hx
l72 = bbq_72.x_logicals

bbq_108 = BivariateBicycle(a, b, 6, 9, 1)
h108 = bbq_108.hx
l108 = bbq_108.x_logicals

bbq_144 = BivariateBicycle(a, b, 6, 12, 1)
h144 = bbq_144.hx
l144 = bbq_144.x_logicals

In [34]:
x, y = Monomial(field3, "x"), Monomial(field3, "y")

a = y**3 + x + x**2
b = x**3 + y + y**2

bbq_72_3 = BivariateBicycle(a, b, 6, 6, 1)
h72_3 = bbq_72.hx
l72_3 = bbq_72.x_logicals

bbq_108_3 = BivariateBicycle(a, b, 6, 9, 1)
h108_3 = bbq_108.hx
l108_3 = bbq_108.x_logicals

bbq_144_3 = BivariateBicycle(a, b, 6, 12, 1)
h144_3 = bbq_144.hx
l144_3 = bbq_144.x_logicals

In [35]:
x, y = Monomial(field5, "x"), Monomial(field5, "y")

a = y**3 + x + x**2
b = x**3 + y + y**2

bbq_72_5 = BivariateBicycle(a, b, 6, 6, 1)
h72_5 = bbq_72.hx
l72_5 = bbq_72.x_logicals

bbq_108_5 = BivariateBicycle(a, b, 6, 9, 1)
h108_5 = bbq_108.hx
l108_5 = bbq_108.x_logicals

bbq_144_5 = BivariateBicycle(a, b, 6, 12, 1)
h144_5 = bbq_144.hx
l144_5 = bbq_144.x_logicals

In [15]:
results = np.flip(simulate(field, h72, l72, max_iter, 20, physical_error))

Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 1
Found 3 / 20, with num_trials : 2
Found 4 / 20, with num_trials : 3
Found 5 / 20, with num_trials : 4
Found 6 / 20, with num_trials : 5
Found 7 / 20, with num_trials : 6
Found 8 / 20, with num_trials : 7
Found 9 / 20, with num_trials : 8
Found 10 / 20, with num_trials : 9
Found 11 / 20, with num_trials : 10
Found 12 / 20, with num_trials : 11
Found 13 / 20, with num_trials : 12
Found 14 / 20, with num_trials : 13
Found 15 / 20, with num_trials : 14
Found 16 / 20, with num_trials : 15
Found 17 / 20, with num_trials : 16
Found 18 / 20, with num_trials : 17
Found 19 / 20, with num_trials : 18
Found 20 / 20, with num_trials : 19
Finished p=0.19952623149688797 with num_trials=20
------------------------------------------------------------------
Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 1
Found 3 / 20, with num_trials : 2
Found 4 / 20, with num_trials : 3
Found 5 / 20, with num_trials : 4
Found 6 / 20

C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:165: RuntimeWarning: divide by zero encountered in divide
  posteriors /= (
C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:142: RuntimeWarning: invalid value encountered in divide
  sub_posteriors = sub_posteriors / posterior
C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:120: RuntimeWarning: invalid value encountered in divide
  sub_convolutions = sub_convolutions / convolution


Found 3 / 20, with num_trials : 5
Found 4 / 20, with num_trials : 7
Found 5 / 20, with num_trials : 8
Found 6 / 20, with num_trials : 9
Found 7 / 20, with num_trials : 10
Found 8 / 20, with num_trials : 12
Found 9 / 20, with num_trials : 16
Found 10 / 20, with num_trials : 17
Found 11 / 20, with num_trials : 19
Found 12 / 20, with num_trials : 20
Found 13 / 20, with num_trials : 22
Found 14 / 20, with num_trials : 23
Found 15 / 20, with num_trials : 26
Found 16 / 20, with num_trials : 27
Found 17 / 20, with num_trials : 29
Found 18 / 20, with num_trials : 30
Found 19 / 20, with num_trials : 31
Found 20 / 20, with num_trials : 33
Finished p=0.09261187281287937 with num_trials=34
------------------------------------------------------------------
Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 4
Found 3 / 20, with num_trials : 7
Found 4 / 20, with num_trials : 10
Found 5 / 20, with num_trials : 15
Found 6 / 20, with num_trials : 16
Found 7 / 20, with num_trials : 21
Foun

In [20]:
results = [1938, 1069, 433, 164, 101, 49, 34, 29, 21, 20]
results

[1938, 1069, 433, 164, 101, 49, 34, 29, 21, 20]

In [8]:
results

array([180,  50,  32,  34,  38,  19,   8,   5,   5,   5])

In [16]:
results = np.flip(simulate(field, h108, l108, max_iter, 20, physical_error))

Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 1
Found 3 / 20, with num_trials : 2
Found 4 / 20, with num_trials : 3
Found 5 / 20, with num_trials : 4
Found 6 / 20, with num_trials : 5
Found 7 / 20, with num_trials : 6
Found 8 / 20, with num_trials : 7
Found 9 / 20, with num_trials : 8
Found 10 / 20, with num_trials : 9
Found 11 / 20, with num_trials : 10
Found 12 / 20, with num_trials : 11
Found 13 / 20, with num_trials : 12
Found 14 / 20, with num_trials : 13
Found 15 / 20, with num_trials : 14
Found 16 / 20, with num_trials : 15
Found 17 / 20, with num_trials : 16
Found 18 / 20, with num_trials : 17
Found 19 / 20, with num_trials : 18
Found 20 / 20, with num_trials : 19
Finished p=0.19952623149688797 with num_trials=20
------------------------------------------------------------------
Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 1
Found 3 / 20, with num_trials : 2
Found 4 / 20, with num_trials : 3
Found 5 / 20, with num_trials : 4
Found 6 / 20

C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:165: RuntimeWarning: divide by zero encountered in divide
  posteriors /= (
C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:142: RuntimeWarning: invalid value encountered in divide
  sub_posteriors = sub_posteriors / posterior
C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:120: RuntimeWarning: invalid value encountered in divide
  sub_convolutions = sub_convolutions / convolution


Found 1 / 20, with num_trials : 12
Found 2 / 20, with num_trials : 15
Found 3 / 20, with num_trials : 16
Found 4 / 20, with num_trials : 19
Found 5 / 20, with num_trials : 22
Found 6 / 20, with num_trials : 23
Found 7 / 20, with num_trials : 26
Found 8 / 20, with num_trials : 39
Found 9 / 20, with num_trials : 43
Found 10 / 20, with num_trials : 45
Found 11 / 20, with num_trials : 50
Found 12 / 20, with num_trials : 59
Found 13 / 20, with num_trials : 65
Found 14 / 20, with num_trials : 69
Found 15 / 20, with num_trials : 70
Found 16 / 20, with num_trials : 75
Found 17 / 20, with num_trials : 78
Found 18 / 20, with num_trials : 82
Found 19 / 20, with num_trials : 89
Found 20 / 20, with num_trials : 94
Finished p=0.07170600970409613 with num_trials=95
------------------------------------------------------------------
UPDATE: Found 0 / 20, with num_trials : 0
Found 1 / 20, with num_trials : 25
Found 2 / 20, with num_trials : 37
Found 3 / 20, with num_trials : 46
UPDATE: Found 3 / 20, wit

C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:146: RuntimeWarning: invalid value encountered in divide
  sub_posteriors / np.sum(sub_posteriors, axis=1)[:, np.newaxis]
C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:165: RuntimeWarning: invalid value encountered in divide
  posteriors /= (


Found 2 / 20, with num_trials : 137
UPDATE: Found 2 / 20, with num_trials : 200
UPDATE: Found 2 / 20, with num_trials : 300
UPDATE: Found 2 / 20, with num_trials : 400
UPDATE: Found 2 / 20, with num_trials : 500
UPDATE: Found 2 / 20, with num_trials : 600
Found 3 / 20, with num_trials : 624
UPDATE: Found 3 / 20, with num_trials : 700
UPDATE: Found 3 / 20, with num_trials : 800
Found 4 / 20, with num_trials : 803
Found 5 / 20, with num_trials : 885
UPDATE: Found 5 / 20, with num_trials : 900
Found 6 / 20, with num_trials : 920
Found 7 / 20, with num_trials : 991
UPDATE: Found 7 / 20, with num_trials : 1000
UPDATE: Found 7 / 20, with num_trials : 1100
UPDATE: Found 7 / 20, with num_trials : 1200
Found 8 / 20, with num_trials : 1262
Found 9 / 20, with num_trials : 1266
UPDATE: Found 9 / 20, with num_trials : 1300
Found 10 / 20, with num_trials : 1399
UPDATE: Found 10 / 20, with num_trials : 1400
Found 11 / 20, with num_trials : 1414
UPDATE: Found 11 / 20, with num_trials : 1500
UPDATE: Fo

In [17]:
results

array([9308, 4829, 2871,  634,  333,   95,   43,   26,   20,   20])

In [18]:
results = np.flip(simulate(field, h144, l144, max_iter, 20, physical_error))

Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 1
Found 3 / 20, with num_trials : 2
Found 4 / 20, with num_trials : 3
Found 5 / 20, with num_trials : 4
Found 6 / 20, with num_trials : 5
Found 7 / 20, with num_trials : 6
Found 8 / 20, with num_trials : 7
Found 9 / 20, with num_trials : 8
Found 10 / 20, with num_trials : 9
Found 11 / 20, with num_trials : 10
Found 12 / 20, with num_trials : 11
Found 13 / 20, with num_trials : 12
Found 14 / 20, with num_trials : 13
Found 15 / 20, with num_trials : 14
Found 16 / 20, with num_trials : 15
Found 17 / 20, with num_trials : 16
Found 18 / 20, with num_trials : 17
Found 19 / 20, with num_trials : 18
Found 20 / 20, with num_trials : 19
Finished p=0.19952623149688797 with num_trials=20
------------------------------------------------------------------
Found 1 / 20, with num_trials : 0
Found 2 / 20, with num_trials : 1
Found 3 / 20, with num_trials : 2
Found 4 / 20, with num_trials : 3
Found 5 / 20, with num_trials : 4
Found 6 / 20

C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:165: RuntimeWarning: divide by zero encountered in divide
  posteriors /= (
C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:142: RuntimeWarning: invalid value encountered in divide
  sub_posteriors = sub_posteriors / posterior
C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:120: RuntimeWarning: invalid value encountered in divide
  sub_convolutions = sub_convolutions / convolution


Found 10 / 20, with num_trials : 9
Found 11 / 20, with num_trials : 10
Found 12 / 20, with num_trials : 11
Found 13 / 20, with num_trials : 13
Found 14 / 20, with num_trials : 14
Found 15 / 20, with num_trials : 16
Found 16 / 20, with num_trials : 17
Found 17 / 20, with num_trials : 18
Found 18 / 20, with num_trials : 19
Found 19 / 20, with num_trials : 20
Found 20 / 20, with num_trials : 21
Finished p=0.11961283330787534 with num_trials=22
------------------------------------------------------------------
UPDATE: Found 0 / 20, with num_trials : 0
Found 1 / 20, with num_trials : 1
Found 2 / 20, with num_trials : 2
Found 3 / 20, with num_trials : 3
Found 4 / 20, with num_trials : 5
Found 5 / 20, with num_trials : 6
Found 6 / 20, with num_trials : 7
Found 7 / 20, with num_trials : 8
Found 8 / 20, with num_trials : 9
Found 9 / 20, with num_trials : 10
Found 10 / 20, with num_trials : 12
Found 11 / 20, with num_trials : 13
Found 12 / 20, with num_trials : 14
Found 13 / 20, with num_trials 

C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:165: RuntimeWarning: invalid value encountered in divide
  posteriors /= (


Found 19 / 20, with num_trials : 1077
UPDATE: Found 19 / 20, with num_trials : 1100
Found 20 / 20, with num_trials : 1139
Finished p=0.033282981394546204 with num_trials=1140
------------------------------------------------------------------
UPDATE: Found 0 / 20, with num_trials : 0
UPDATE: Found 0 / 20, with num_trials : 100
UPDATE: Found 0 / 20, with num_trials : 200
UPDATE: Found 0 / 20, with num_trials : 300
UPDATE: Found 0 / 20, with num_trials : 400
UPDATE: Found 0 / 20, with num_trials : 500
Found 1 / 20, with num_trials : 517
Found 2 / 20, with num_trials : 600
UPDATE: Found 2 / 20, with num_trials : 700
UPDATE: Found 2 / 20, with num_trials : 800
Found 3 / 20, with num_trials : 863
UPDATE: Found 3 / 20, with num_trials : 900
Found 4 / 20, with num_trials : 973
UPDATE: Found 4 / 20, with num_trials : 1000
UPDATE: Found 4 / 20, with num_trials : 1100
Found 5 / 20, with num_trials : 1110
UPDATE: Found 5 / 20, with num_trials : 1200
Found 6 / 20, with num_trials : 1231
Found 7 / 2

C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:146: RuntimeWarning: invalid value encountered in divide
  sub_posteriors / np.sum(sub_posteriors, axis=1)[:, np.newaxis]


Found 16 / 20, with num_trials : 1986
UPDATE: Found 16 / 20, with num_trials : 2000
UPDATE: Found 16 / 20, with num_trials : 2100
Found 17 / 20, with num_trials : 2121
UPDATE: Found 17 / 20, with num_trials : 2200
Found 18 / 20, with num_trials : 2285
Found 19 / 20, with num_trials : 2295
UPDATE: Found 19 / 20, with num_trials : 2300
UPDATE: Found 19 / 20, with num_trials : 2400
UPDATE: Found 19 / 20, with num_trials : 2500
UPDATE: Found 19 / 20, with num_trials : 2600
Found 20 / 20, with num_trials : 2602
Finished p=0.025769803745148787 with num_trials=2603
------------------------------------------------------------------
UPDATE: Found 0 / 20, with num_trials : 0
UPDATE: Found 0 / 20, with num_trials : 100
Found 1 / 20, with num_trials : 146
UPDATE: Found 1 / 20, with num_trials : 200
Found 2 / 20, with num_trials : 206
UPDATE: Found 2 / 20, with num_trials : 300
UPDATE: Found 2 / 20, with num_trials : 400
Found 3 / 20, with num_trials : 429
UPDATE: Found 3 / 20, with num_trials : 50

In [19]:
results

array([2968, 2603, 1140,  447,  272,   83,   32,   22,   20,   20])

In [19]:
x_order = ['Idle', 0, 3, 1, 2]
z_order = [0, 3, 1, 2, 'Idle']
max_iter = 100

num_cycles = 5
circ = bb25.construct_sm_circuit(x_order, z_order)

In [15]:
circ_physical_error = np.flip(np.logspace(-2.3, -2, 10))
circ_physical_error

array([0.01      , 0.00926119, 0.00857696, 0.00794328, 0.00735642,
       0.00681292, 0.00630957, 0.00584341, 0.0054117 , 0.00501187])

In [20]:
results = circ_simulate(Field(2), bb25, circ, num_cycles, max_iter, 5, circ_physical_error)

Constructing decoding matrix for p=0.01...
Constructed decoding matrix for p=0.01


C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:142: RuntimeWarning: invalid value encountered in divide
  sub_posteriors = sub_posteriors / posterior
C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:165: RuntimeWarning: divide by zero encountered in divide
  posteriors /= (
C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:165: RuntimeWarning: invalid value encountered in divide
  posteriors /= (
C:\Users\elean\Documents\Quantum-Computing-PhD\ITP\IP-BBQ\qudit-bivariate-bicycle\bbq\decoder.py:120: RuntimeWarning: invalid value encountered in divide
  sub_convolutions = sub_convolutions / convolution


Found 1 / 5, with num_trials : 0
Found 2 / 5, with num_trials : 1
Found 3 / 5, with num_trials : 2
Found 4 / 5, with num_trials : 5
Found 5 / 5, with num_trials : 7
Finished p=0.01 with num_trials=8
------------------------------------------------------------------
Constructing decoding matrix for p=0.009261187281287938...
Constructed decoding matrix for p=0.009261187281287938
Found 1 / 5, with num_trials : 0
Found 2 / 5, with num_trials : 3
Found 3 / 5, with num_trials : 4
Found 4 / 5, with num_trials : 5
Found 5 / 5, with num_trials : 7
Finished p=0.009261187281287938 with num_trials=8
------------------------------------------------------------------
Constructing decoding matrix for p=0.008576958985908946...
Constructed decoding matrix for p=0.008576958985908946
UPDATE: Found 0 / 5, with num_trials : 0
Found 1 / 5, with num_trials : 1
Found 2 / 5, with num_trials : 3
Found 3 / 5, with num_trials : 5
Found 4 / 5, with num_trials : 6
Found 5 / 5, with num_trials : 7
Finished p=0.00857

In [21]:
# d = 5

results

[8, 8, 8, 6, 10, 13, 8, 14, 8, 8]

In [ ]:
# d = 3

results

[12, 23, 22, 22, 16, 20, 6, 21, 28, 28]

In [18]:
5/28

0.17857142857142858

In [ ]:
results = np.flip(simulate(5, h55, l55, max_iter, 50, ext_physical_error))

In [ ]:
results

In [ ]:
results = np.flip(simulate(5, h57, l57, max_iter, 50, ext_physical_error))

In [ ]:
results

In [ ]:
results = np.flip(simulate(5, h5, l5, max_iter, 50, ext_physical_error))

In [ ]:
results

In [ ]:
results = np.flip(simulate(5, h55, l55, max_iter, 50, ext_physical_error))

In [ ]:
results

In [ ]:
results = np.flip(simulate(5, h57, l57, max_iter, 50, ext_physical_error))

In [ ]:
results